# Notebook 02: Análisis de Hot Spots

Este notebook implementa el análisis de **Hot Spots (Getis-Ord Gi*)** para identificar
zonas de alta y baja concentración de edificaciones en Isla de Pascua.

**Contenido:**
1. Carga de datos
2. Creación de grilla de análisis
3. Cálculo de Getis-Ord Gi*
4. Visualización de Hot/Cold Spots
5. Interpretación de resultados

In [ ]:
# Importar librerías
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import box
from scipy.spatial.distance import cdist
import warnings
warnings.filterwarnings('ignore')

# Configuración
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)

print("Librerías cargadas correctamente")

## 1. Cargar Datos

In [ ]:
# Rutas de datos
DATA_PATH = '../data/raw/isla_de_pascua'

# Cargar datasets
boundary = gpd.read_file(f'{DATA_PATH}/isla_de_pascua_boundary.geojson')
buildings = gpd.read_file(f'{DATA_PATH}/isla_de_pascua_buildings.geojson')

print(f"Límite de la isla cargado: {len(boundary)} polígonos")
print(f"Edificios cargados: {len(buildings)} registros")

# Proyectar a UTM
CRS_UTM = "EPSG:32712"  # UTM Zone 12S para Isla de Pascua
boundary_utm = boundary.to_crs(CRS_UTM)
buildings_utm = buildings.to_crs(CRS_UTM)

# Calcular área de edificios
buildings_utm['area_m2'] = buildings_utm.geometry.area
print(f"\nÁrea total construida: {buildings_utm['area_m2'].sum()/10000:.2f} ha")

## 2. Crear Grilla de Análisis

In [ ]:
# Crear grilla hexagonal/cuadrada para análisis
cell_size = 200  # metros

# Bounds de la isla
minx, miny, maxx, maxy = boundary_utm.total_bounds

# Crear celdas
cells = []
x = minx
while x < maxx:
    y = miny
    while y < maxy:
        cell = box(x, y, x + cell_size, y + cell_size)
        cells.append(cell)
        y += cell_size
    x += cell_size

# Crear GeoDataFrame
grid = gpd.GeoDataFrame({'geometry': cells}, crs=CRS_UTM)

# Filtrar celdas dentro de la isla
grid = grid[grid.intersects(boundary_utm.unary_union)].reset_index(drop=True)
grid['cell_id'] = range(len(grid))

print(f"Grilla creada: {len(grid)} celdas de {cell_size}m x {cell_size}m")

In [ ]:
# Contar edificios por celda
grid['building_count'] = 0
grid['building_area'] = 0.0

for idx, cell in grid.iterrows():
    buildings_in_cell = buildings_utm[buildings_utm.intersects(cell.geometry)]
    grid.loc[idx, 'building_count'] = len(buildings_in_cell)
    grid.loc[idx, 'building_area'] = buildings_in_cell['area_m2'].sum()

print(f"Celdas con edificios: {(grid['building_count'] > 0).sum()}")
print(f"Máximo de edificios en una celda: {grid['building_count'].max()}")

## 3. Implementación de Getis-Ord Gi*

In [ ]:
def create_distance_weights(gdf, threshold=None):
    """
    Crear matriz de pesos espaciales basada en distancia.
    """
    # Obtener centroides
    centroids = np.column_stack([gdf.geometry.centroid.x, gdf.geometry.centroid.y])
    
    # Calcular matriz de distancias
    distances = cdist(centroids, centroids)
    
    # Si no hay threshold, usar distancia promedio al vecino más cercano * 1.5
    if threshold is None:
        np.fill_diagonal(distances, np.inf)
        min_dists = distances.min(axis=1)
        threshold = min_dists.mean() * 1.5
        np.fill_diagonal(distances, 0)
    
    # Crear matriz de pesos binaria
    W = (distances <= threshold).astype(float)
    np.fill_diagonal(W, 0)  # Sin auto-vecindad
    
    return W, threshold


def getis_ord_gi_star(values, W):
    """
    Calcular estadístico Getis-Ord Gi* para hot spot analysis.
    """
    n = len(values)
    x_mean = values.mean()
    x_std = values.std()
    
    # Incluir la propia observación en Gi*
    W_star = W.copy()
    np.fill_diagonal(W_star, 1)
    
    # Calcular Gi* para cada observación
    gi_star = np.zeros(n)
    z_scores = np.zeros(n)
    
    for i in range(n):
        wi = W_star[i, :]
        sum_wij = wi.sum()
        sum_wij_xj = (wi * values).sum()
        sum_wij2 = (wi ** 2).sum()
        
        # Numerador
        numerator = sum_wij_xj - x_mean * sum_wij
        
        # Denominador
        s = np.sqrt(((values ** 2).sum() / n) - x_mean ** 2)
        denominator = s * np.sqrt((n * sum_wij2 - sum_wij ** 2) / (n - 1))
        
        if denominator > 0:
            z_scores[i] = numerator / denominator
        else:
            z_scores[i] = 0
    
    return z_scores


def classify_hotspots(z_scores):
    """
    Clasificar zonas según Z-scores.
    """
    classification = np.zeros(len(z_scores), dtype=int)
    
    # Hot spots
    classification[z_scores > 2.58] = 1   # 99% confianza
    classification[(z_scores > 1.96) & (z_scores <= 2.58)] = 2   # 95%
    classification[(z_scores > 1.65) & (z_scores <= 1.96)] = 3   # 90%
    
    # Cold spots
    classification[z_scores < -2.58] = -1  # 99% confianza
    classification[(z_scores < -1.96) & (z_scores >= -2.58)] = -2  # 95%
    classification[(z_scores < -1.65) & (z_scores >= -1.96)] = -3  # 90%
    
    return classification

print("Funciones de Getis-Ord Gi* definidas")

In [ ]:
# Calcular análisis de Hot Spots
print("Calculando matriz de pesos espaciales...")
W, threshold = create_distance_weights(grid)
print(f"Threshold de distancia: {threshold:.0f} metros")

# Valores a analizar
values = grid['building_count'].values.astype(float)

print("\nCalculando Getis-Ord Gi*...")
z_scores = getis_ord_gi_star(values, W)

# Clasificar
classification = classify_hotspots(z_scores)

# Agregar al GeoDataFrame
grid['z_score'] = z_scores
grid['hotspot_class'] = classification

# Etiquetas
labels = {
    1: 'Hot Spot (99%)',
    2: 'Hot Spot (95%)',
    3: 'Hot Spot (90%)',
    0: 'No Significativo',
    -3: 'Cold Spot (90%)',
    -2: 'Cold Spot (95%)',
    -1: 'Cold Spot (99%)'
}
grid['hotspot_label'] = grid['hotspot_class'].map(labels)

print("\nDistribución de Hot/Cold Spots:")
print(grid['hotspot_label'].value_counts())

## 4. Visualización de Resultados

In [ ]:
# Mapa de Hot Spots
fig, axes = plt.subplots(1, 2, figsize=(16, 8))

# Mapa 1: Densidad de edificios
ax1 = axes[0]
boundary_utm.plot(ax=ax1, color='lightgray', edgecolor='black', linewidth=0.5)
grid[grid['building_count'] > 0].plot(
    ax=ax1,
    column='building_count',
    cmap='YlOrRd',
    legend=True,
    legend_kwds={'label': 'Número de Edificios'}
)
ax1.set_title('Densidad de Edificaciones', fontsize=14, fontweight='bold')
ax1.set_axis_off()

# Mapa 2: Hot Spots
ax2 = axes[1]
boundary_utm.plot(ax=ax2, color='lightgray', edgecolor='black', linewidth=0.5)

# Colores para hot/cold spots
colors = {
    1: '#b2182b',   # Hot Spot 99% - Rojo oscuro
    2: '#ef8a62',   # Hot Spot 95% - Rojo claro
    3: '#fddbc7',   # Hot Spot 90% - Rosa
    0: '#f7f7f7',   # No significativo - Gris claro
    -3: '#d1e5f0',  # Cold Spot 90% - Celeste
    -2: '#67a9cf',  # Cold Spot 95% - Azul claro
    -1: '#2166ac'   # Cold Spot 99% - Azul oscuro
}

# Plotear cada categoría
for class_val, color in colors.items():
    subset = grid[grid['hotspot_class'] == class_val]
    if len(subset) > 0:
        subset.plot(ax=ax2, color=color, edgecolor='gray', linewidth=0.1,
                   label=labels[class_val])

ax2.set_title('Análisis de Hot Spots (Getis-Ord Gi*)', fontsize=14, fontweight='bold')
ax2.legend(loc='lower left', fontsize=9)
ax2.set_axis_off()

plt.tight_layout()
plt.savefig('../outputs/hotspots_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✓ Mapa guardado en outputs/hotspots_analysis.png")

In [ ]:
# Histograma de Z-scores
fig, ax = plt.subplots(figsize=(10, 6))

ax.hist(z_scores, bins=30, color='steelblue', edgecolor='white', alpha=0.7)

# Líneas de significancia
ax.axvline(x=1.96, color='red', linestyle='--', label='95% (z=1.96)')
ax.axvline(x=-1.96, color='blue', linestyle='--', label='95% (z=-1.96)')
ax.axvline(x=2.58, color='darkred', linestyle=':', label='99% (z=2.58)')
ax.axvline(x=-2.58, color='darkblue', linestyle=':', label='99% (z=-2.58)')

ax.set_xlabel('Z-Score (Gi*)', fontsize=12)
ax.set_ylabel('Frecuencia', fontsize=12)
ax.set_title('Distribución de Z-Scores del Análisis Gi*', fontsize=14, fontweight='bold')
ax.legend(loc='upper right')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../outputs/hotspots_zscore_distribution.png', dpi=150)
plt.show()

## 5. Interpretación de Resultados

In [ ]:
# Estadísticas de Hot Spots
print("="*60)
print("RESUMEN DEL ANÁLISIS DE HOT SPOTS")
print("="*60)

# Contar por categoría
hot_99 = (grid['hotspot_class'] == 1).sum()
hot_95 = (grid['hotspot_class'] == 2).sum()
hot_90 = (grid['hotspot_class'] == 3).sum()
cold_90 = (grid['hotspot_class'] == -3).sum()
cold_95 = (grid['hotspot_class'] == -2).sum()
cold_99 = (grid['hotspot_class'] == -1).sum()
no_sig = (grid['hotspot_class'] == 0).sum()

total_hot = hot_99 + hot_95 + hot_90
total_cold = cold_99 + cold_95 + cold_90

print(f"\nHot Spots (alta concentración):")
print(f"  - 99% confianza: {hot_99} celdas")
print(f"  - 95% confianza: {hot_95} celdas")
print(f"  - 90% confianza: {hot_90} celdas")
print(f"  Total: {total_hot} celdas ({100*total_hot/len(grid):.1f}%)")

print(f"\nCold Spots (baja concentración):")
print(f"  - 90% confianza: {cold_90} celdas")
print(f"  - 95% confianza: {cold_95} celdas")
print(f"  - 99% confianza: {cold_99} celdas")
print(f"  Total: {total_cold} celdas ({100*total_cold/len(grid):.1f}%)")

print(f"\nNo significativo: {no_sig} celdas ({100*no_sig/len(grid):.1f}%)")

print("\n" + "="*60)
print("INTERPRETACIÓN")
print("="*60)
print("""
Los Hot Spots identificados corresponden principalmente a la zona
urbana de Hanga Roa, único centro poblado de la isla. La alta
concentración de edificaciones en esta área refleja el patrón
histórico de asentamiento de la población Rapa Nui.

Las zonas clasificadas como Cold Spots corresponden a áreas
periféricas con muy baja o nula densidad de construcciones,
incluyendo zonas de conservación y sitios arqueológicos.

Este análisis es útil para:
- Planificación territorial
- Identificación de presión urbana
- Gestión de zonas de conservación
""")

In [ ]:
# Guardar resultados
output_gdf = grid[['cell_id', 'building_count', 'building_area', 'z_score', 
                   'hotspot_class', 'hotspot_label', 'geometry']].copy()

output_gdf.to_file('../outputs/hotspots_results.geojson', driver='GeoJSON')
print("\n✓ Resultados guardados en outputs/hotspots_results.geojson")

# Resumen CSV
summary = grid.groupby('hotspot_label').agg({
    'cell_id': 'count',
    'building_count': 'sum',
    'building_area': 'sum',
    'z_score': 'mean'
}).round(2)
summary.columns = ['n_celdas', 'total_edificios', 'area_total_m2', 'z_score_promedio']
summary.to_csv('../outputs/hotspots_summary.csv')
print("✓ Resumen guardado en outputs/hotspots_summary.csv")

display(summary)